In [2]:
import import_ipynb
from maze_common import Maze, shortestPathSearch
import random as rnd
import numpy as np
import copy

importing Jupyter notebook from maze_common.ipynb
[[ 0  0 -1  0  0]
 [ 0 -1 -1  0 -1]
 [ 0 -1 -1 -1  0]
 [-1  0  0  0 -1]
 [ 0 -1  0  0  0]]
Is solvable?  False


In [3]:
def thinMaze(maze, fractionRemove):
    numRemove = round(fractionRemove * maze.obstacles.size / 2)
    thinMaze = copy.deepcopy(maze)
    for i in range(0,numRemove):
        indexRemove = int(rnd.random() * thinMaze.obstacles.size / 2)
        obstacleX, obstacleY = thinMaze.obstacles[indexRemove]
        thinMaze.obstacles = np.delete(thinMaze.obstacles,indexRemove, axis=0)
        thinMaze.board[obstacleX][obstacleY] = 0
    return thinMaze
    
maze = Maze(5, .5)
thinnedMaze = thinMaze(maze, .6)
print("Original obstacles:\n", maze.obstacles)
print("Thinned obstacles:\n", thinnedMaze.obstacles)

Original obstacles:
 [[0 0]
 [0 3]
 [1 0]
 [1 1]
 [1 3]
 [1 4]
 [2 0]
 [2 4]
 [3 0]
 [3 2]
 [3 3]
 [4 2]
 [4 3]]
Thinned obstacles:
 [[1 0]
 [1 4]
 [3 2]
 [3 3]
 [4 2]]
